# Project Assignment 1#
By: Qingwen Jia, Jiale Qian,Vinson Liang, Joses Ng, Natalie Choy


In [1]:
import pandas as pd
df_indicators = pd.read_csv('country_indicators.csv')
import plotly.graph_objects as go
import plotly.io as pio
df_preds = pd.read_csv('test_predictions.csv')
import numpy as np
from sklearn import metrics
import seaborn as sns

# 1. Create the Prediciton Probability "Error" results for the xgboost and ffnn models analagously to the transformer model produced above.

In [2]:
df_preds['ffnn_probability_prediction_error'] = np.abs(df_preds['y_true_ffnn'].astype(float) - df_preds['y_pred_proba_ffnn'])
df_preds[['y_true_ffnn','y_pred_proba_ffnn','ffnn_probability_prediction_error']]

,y_true_ffnn,y_pred_proba_ffnn,ffnn_probability_prediction_error
0,False,0.409958,0.409958
1,False,0.406696,0.406696
2,False,0.545236,0.545236
3,False,0.534560,0.534560
4,True,0.538583,0.461417
...,...,...,...
359,False,0.291874,0.291874
360,False,0.300321,0.300321
361,False,0.335496,0.335496
362,False,0.324000,0.324000


In [3]:
df_preds['xgboost_probability_prediction_error'] = np.abs(df_preds['y_true_xgboost'].astype(float) - df_preds['y_pred_proba_xgboost'])
df_preds[['y_true_xgboost','y_pred_proba_xgboost','xgboost_probability_prediction_error']]

,y_true_xgboost,y_pred_proba_xgboost,xgboost_probability_prediction_error
0,False,0.066500,0.066500
1,False,0.099643,0.099643
2,True,0.704086,0.295914
3,True,0.638444,0.361556
4,False,0.608380,0.608380
...,...,...,...
359,False,0.079453,0.079453
360,False,0.060189,0.060189
361,True,0.697625,0.302375
362,False,0.729246,0.729246


# 2. Create a bootsrap confidence interval for the average Prediction Probability "Error" for one of these models using all the data.

In [4]:
n = 364
repetitions = 10000
my_bootstrapped_population_sample_means = np.zeros(repetitions)
for i in range(repetitions):
    my_bootstrapped_population_sample_n364 = \
        np.random.choice(df_preds['xgboost_probability_prediction_error'], size=364, replace=True)
    my_bootstrapped_population_sample_means[i] = \
        my_bootstrapped_population_sample_n364.mean()

In [5]:
#95% Confidence Interval
np.quantile(my_bootstrapped_population_sample_means, [0.05, 0.95])

array([0.42609405, 0.46481134])

# 3. Create the Prediction Classification "Correctness" results of "correct" and "incorrect" predictions for the transformer, xgboost and ffnn models; or, an alternative "either/or" breakdown of interest (such as "wrongly predicted no escalation" versus all the other categories combined).

In [6]:
# transformer
threshold = 0.5
df_preds['transformer_classifcation_performance_outcome'] = None

tmp = df_preds['transformer_classifcation_performance_outcome'].copy()
TP_pos_pred_correct = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer>threshold)
tmp[TP_pos_pred_correct] = "correctly predicted escalation"
TN_neg_pred_correct = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer<=threshold)
tmp[TN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer>threshold)
tmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer<=threshold)
tmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['transformer_classifcation_performance_outcome'] = tmp
df_preds[['y_true_transformer','y_pred_transformer','transformer_classifcation_performance_outcome']][30:50]

,y_true_transformer,y_pred_transformer,transformer_classifcation_performance_outcome
30,True,True,correctly predicted escalation
31,False,True,wrongly predicted escalation
32,False,True,wrongly predicted escalation
33,False,True,wrongly predicted escalation
34,False,False,correctly predicted no escalation
35,False,True,wrongly predicted escalation
36,False,False,correctly predicted no escalation
37,False,False,correctly predicted no escalation
38,False,True,wrongly predicted escalation
39,False,True,wrongly predicted escalation


In [7]:
# xgboost
threshold = 0.5
df_preds['xgboost_classifcation_performance_outcome'] = None

tmp = df_preds['xgboost_classifcation_performance_outcome'].copy()
TP_pos_pred_correct = df_preds.y_true_xgboost & (df_preds.y_pred_proba_xgboost>threshold)
tmp[TP_pos_pred_correct] = "correctly predicted escalation"
TN_neg_pred_correct = (~df_preds.y_true_xgboost) & (df_preds.y_pred_proba_xgboost<=threshold)
tmp[TN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_xgboost) & (df_preds.y_pred_proba_xgboost>threshold)
tmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_xgboost & (df_preds.y_pred_proba_xgboost<=threshold)
tmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['xgboost_classifcation_performance_outcome'] = tmp
df_preds[['y_true_xgboost','y_pred_xgboost','xgboost_classifcation_performance_outcome']][30:50]

,y_true_xgboost,y_pred_xgboost,xgboost_classifcation_performance_outcome
30,False,True,wrongly predicted escalation
31,False,True,wrongly predicted escalation
32,False,True,wrongly predicted escalation
33,False,True,wrongly predicted escalation
34,False,False,correctly predicted no escalation
35,False,True,wrongly predicted escalation
36,False,True,wrongly predicted escalation
37,False,True,wrongly predicted escalation
38,True,True,correctly predicted escalation
39,False,True,wrongly predicted escalation


In [8]:
# ffnn
threshold = 0.5
df_preds['ffnn_classifcation_performance_outcome'] = None

tmp = df_preds['ffnn_classifcation_performance_outcome'].copy()
TP_pos_pred_correct = df_preds.y_true_ffnn & (df_preds.y_pred_proba_ffnn>threshold)
tmp[TP_pos_pred_correct] = "correctly predicted escalation"
TN_neg_pred_correct = (~df_preds.y_true_ffnn) & (df_preds.y_pred_proba_ffnn<=threshold)
tmp[TN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_ffnn) & (df_preds.y_pred_proba_ffnn>threshold)
tmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_ffnn & (df_preds.y_pred_proba_ffnn<=threshold)
tmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['ffnn_classifcation_performance_outcome'] = tmp
df_preds[['y_true_ffnn','y_pred_ffnn','ffnn_classifcation_performance_outcome']][30:50]

,y_true_ffnn,y_pred_ffnn,ffnn_classifcation_performance_outcome
30,True,True,correctly predicted escalation
31,False,False,correctly predicted no escalation
32,False,True,wrongly predicted escalation
33,False,True,wrongly predicted escalation
34,False,True,wrongly predicted escalation
35,False,True,wrongly predicted escalation
36,False,True,wrongly predicted escalation
37,False,False,correctly predicted no escalation
38,False,False,correctly predicted no escalation
39,False,False,correctly predicted no escalation


# 4. Perform a one sample hypothesis test of the proportion of a specific Prediction Classification "Correctness" category for another of these models using all the data.

> $H_0: p=0.25$

> $H_a: p \neq 0.25$

In [9]:
from scipy import stats
p_value= np.round(stats.ttest_1samp(df_preds['transformer_classifcation_performance_outcome'] == 'wrongly predicted escalation', keepdims=False, popmean = 0.25)[1], 3)
p_value

0.004

In [10]:
p_value= np.round(stats.ttest_1samp(df_preds['transformer_classifcation_performance_outcome'] == 'correctly predicted escalation', keepdims=False, popmean = 0.25)[1], 3)
p_value

0.0

In [11]:
p_value= np.round(stats.ttest_1samp(df_preds['transformer_classifcation_performance_outcome'] == 'correctly predicted no escalation', keepdims=False, popmean = 0.25)[1], 3)
p_value

0.0

In [12]:
p_value= np.round(stats.ttest_1samp(df_preds['transformer_classifcation_performance_outcome'] == 'wrongly predicted no escalation', keepdims=False, popmean = 0.25)[1], 3)
p_value

0.0

> For transformer predition classification "correctness", the p_value calculated is 0.004 and 0.0 which is between strong and very strong evidence against the null hypothesis. Thus, we could reject the null hypothesis.

# 5. Consider the "Progress Indicators" data and use "boolean selection" with one (or more) of the columns to restrict the data to a subset (of rows) of data and repeat either of the (confidence interval and hypothesis testing) analyses above but this time instead only using this specified subset of countries.

In [13]:
#data wraggling
only_2 = df_indicators.loc[:,["fsi_category","fsi_x1:_external_intervention"]]
onlyalert = only_2.dropna()
onlyalert.groupby("fsi_category").mean()

,fsi_x1:_external_intervention
fsi_category,
Alert,7.975862
Stable,3.772340
Sustainable,0.794444
Warning,5.613580


In [19]:
# bootstap confidence interval
no_sample = 10000
sample_size = onlyalert.shape[0]
np.random.seed(800)
bootstrap_mean = []
for i in range(no_sample):
    bootstrap_sample = np.random.choice((onlyalert['fsi_category'] == "Alert"),replace=True,size = sample_size)
    bootstrap_mean += [np.mean(bootstrap_sample)]
    
np.quantile(bootstrap_mean,[0.05,0.95])
    

array([0.12      , 0.21142857])

# 6 and 7. Create a two-sample bootstrap confidence interval and perform a hypothesis test comparing the performance of a single model for the data subset created above versus the remaining data not included in that data subset.

In [ ]:
#data wraggling
only_2 = df_indicators.loc[:,["fsi_category","fsi_x1:_external_intervention"]]
onlyalert = only_2.dropna()
onlyalert.groupby("fsi_category").mean()
only_2_with4results = only_2.drop(only_2[only_2['fsi_category']== 'Warning'].index)
only_2_with3results = only_2_with4results.dropna()
only_2_with2results = only_2_with3results .drop(only_2_with3results [only_2_with3results ['fsi_category']== 'Sustainable'].index)
only_2_with2results = only_2_with2results.rename(columns={'fsi_x1:_external_intervention': 'external_intervention'})
only_2_with2results

In [ ]:
#bootstrapping fo
np.random.seed(800)
no_of_samples = 1000
stimulated_value = []
bootstrap_intervention_value = only_2_with2results.external_intervention.values.copy()
simdata = only_2_with2results.copy()
for i in range(no_of_samples):
    bootstrap_intervention_value[only_2_with2results.fsi_category == "Stable"] = only_2_with2results.external_intervention[only_2_with2results.fsi_category == "Stable"].sample(frac=1,replace = True).values
    
    bootstrap_intervention_value[only_2_with2results.fsi_category == "Alert"] = only_2_with2results.external_intervention[only_2_with2results.fsi_category == "Alert"].sample(frac=1,replace = True).values
    
    simdata['external_intervention'] = bootstrap_intervention_value
    
    simvalue = np.diff(simdata.groupby('fsi_category').mean().values.flatten())[0]
    stimulated_value += [simvalue]

In [ ]:
#finding the 90% confidence interval of the stimulated samples
np.quantile(stimulated_value,[0.05,0.95])


> $H_0: mean of alert is the same as the mean of stable$

> $H_a: mean of alert is different from the mean of stable$

In [ ]:
observed_diff = np.diff(only_2_with2results.groupby('fsi_category').mean().values.flatten())[0]
observed_diff

In [ ]:
#permutation shuffling for hypothesis testing
np.random.seed(800)
no_of_samples = 1000
stimulated_value_underH0 = []

simdata = only_2_with2results.copy()
for i in range(no_of_samples):
    simdata['fsi_category'] = only_2_with2results['fsi_category'] .sample(frac=1,replace = False).values
    simvalue = np.diff(simdata.groupby('fsi_category').mean().values.flatten())[0]
    stimulated_value_underH0 += [simvalue]
    
extreme_neg = ((stimulated_value_underH0) <= observed_diff).sum()
p_value = extreme_neg / no_of_samples
p_value

> In conclusion, the p_value calculated is 0.0 which is smaller than 0.001. This indicates there is very strong evidence against the null hypothesis. Thus, we fail to reject the null hypothesis and we can find that the external_intervention point mean of the alert category in fsi_category is different from that of the stable category. 

# 8 and 9. Create a bootstrap confidence interval and a hypothesis test comparing the performance of two the models across all the data on the basis of a "paired" sample analysis (by transforming the paired sample into a single difference sample).

> $H_0$: mean of life expectancy at birth years_1970 is the same as the mean of life expectancy at birth $years_{2000}$


> $H_a$: mean of life expectancy at birth years_1970 is different from the mean of life expectancy at birth $years_{2000}$

In [25]:
df_test = pd.read_csv('test_predictions.csv')
df_1970 = df_indicators['sowc_demographics__life-expectancy-at-birth-years_1970'].dropna()
df_2000 = df_indicators['sowc_demographics__life-expectancy-at-birth-years_2000-0'].dropna()
z = df_2000 - df_1970
reps = 1000
observed = z.mean()
null = 0
sample_size = 175
all_samples = np.zeros(reps)
for i in range(reps):
    bootstrap = np.random.choice(z, replace=True, size=sample_size)
    all_samples[i] = np.mean(bootstrap)  
my_bootstrapped_ConfidenceInterval = np.quantile(all_samples,[0.05,0.95])
p_value = (abs(bootstrap - null) >= abs(observed - null)).sum()/1000

print(p_value, my_bootstrapped_ConfidenceInterval)


0.08 [8.13654997 9.70038523]


In [34]:
from scipy import stats
p_value = stats.wilcoxon(df_2000, df_1970)
p_value

WilcoxonResult(statistic=360.0, pvalue=1.9733287695870795e-32)

> As the p value is bewteen 0.1 and 0.05, we have weak evidence against the null hypothesis. Hence, we don't have sufficient evidence to reject the null hypothesis.

> $H_0$: mean of indicator for economy is the same as the indicator for public services

> $H_a$: mean of indicator for economy is the different from the indicator for public services


In [21]:
economy = df_indicators['fsi_e1:_economy'].dropna()
public_service = df_indicators['fsi_p2:_public_services'].dropna()
public_service

0      10.0
1       3.8
2       5.0
4       8.8
6       3.8
       ... 
197     3.8
198     9.6
199     8.1
200     8.8
201     4.8
Name: fsi_p2:_public_services, Length: 175, dtype: float64

In [26]:

economy = df_indicators['fsi_e1:_economy'].dropna()
public_service = df_indicators['fsi_p2:_public_services'].dropna()
z = economy - public_service
reps = 1000
observed = z.mean()
null = 0
sample_size = 175
all_samples = np.zeros(reps)
for i in range(reps):
    bootstrap = np.random.choice(z, replace=True, size=sample_size)
    all_samples[i] = np.mean(bootstrap)  
my_bootstrapped_ConfidenceInterval = np.quantile(all_samples,[0.05,0.95])
p_value = (abs(bootstrap - null) >= abs(observed - null)).sum()/1000
print(p_value, my_bootstrapped_ConfidenceInterval)

0.158 [0.04451429 0.42002857]


> As the p value is bigger than 0.1, we have no evidence against the null hypothesis. Hence, we don't have evidence to reject the null hypothesis.